In [5]:
FOLDER = 'resources/'

import numpy as np
import pandas as pd
import time, datetime
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile

#%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rc('font', family='Times New Roman')

pd.set_option('max_rows', 50)

time_format = '%b %d, %H:%M'

start_time = time.time()
current_time = pd.read_csv(FOLDER + 'current_time.csv').current_time[0]
twr          = pd.read_csv(FOLDER + 'team_work_region.csv', converters={'twr':str})
links        = pd.read_csv(FOLDER + 'link.csv')
stations     = pd.read_csv(FOLDER + 'station.csv', converters={'station':str})
train_info   = pd.read_csv(FOLDER + 'train_info.csv', converters={'train': str, 'st_from':str, 'st_to':str, 'oper_location':str,
                                                                 'st_from':str, 'st_to':str})
train_plan   = pd.read_csv(FOLDER + 'slot_train.csv', converters={'train': str, 'st_from':str, 'st_to':str})
loco_info    = pd.read_csv(FOLDER + 'loco_attributes.csv', converters={'train':str, 'loco':str, 'depot':str,
                                                                      'st_from':str, 'st_to':str})
loco_plan    = pd.read_csv(FOLDER + 'slot_loco.csv', converters={'train':str, 'loco':str, 'st_from':str, 'st_to':str})
team_info    = pd.read_csv(FOLDER + 'team_attributes.csv', converters={'team':str,'depot':str, 'oper_location':str, \
                                                                 'st_from':str, 'st_to':str, 'loco':str, 'depot_st':str})
team_plan    = pd.read_csv(FOLDER + 'slot_team.csv', converters={'team':str,'loco':str, 'st_from':str, 'st_to':str})
loco_series  = pd.read_csv(FOLDER + 'loco_series.csv')

team_info.regions = team_info.regions.apply(literal_eval)
st_names = stations[['station', 'name', 'esr']].drop_duplicates().set_index('station')
print('Planning start time: %s (%d)' % (time.strftime(time_format, time.localtime(current_time)), current_time))

Planning start time: Jun 30, 16:32 (1467293550)


In [6]:
# Мержим таблицы _plan и _info для поездов, локомотивов и бригад
# Добавляем во все таблицы названия станций на маршруте и времена отправления/прибытия в читабельном формате

def add_info(df):    
    if 'st_from' in df.columns:
        df['st_from_name'] = df.st_from.map(st_names.name)
    if 'st_to' in df.columns:
        df['st_to_name'] = df.st_to.map(st_names.name)
    if 'time_start' in df.columns:
        df['time_start_norm'] = df.time_start.apply(lambda x: time.strftime(time_format, time.localtime(x)))
    if 'time_end' in df.columns:
        df['time_end_norm'] = df.time_end.apply(lambda x: time.strftime(time_format, time.localtime(x)))
    if 'oper_location' in df.columns:
        df['oper_location_name'] = df.oper_location.map(st_names.name)    
        df.oper_location_name.fillna(0, inplace=True)
    if ('oper_location' in df.columns) & ('st_from' in df.columns) & ('st_to' in df.columns):        
        df['loc_name'] = df.oper_location_name
        df.loc[df.loc_name == 0, 'loc_name'] = df.st_from_name + ' - ' + df.st_to_name
    
add_info(train_plan)
add_info(loco_plan)
add_info(team_plan)
add_info(loco_info)
add_info(team_info)
add_info(train_info)
train_plan = train_plan.merge(train_info, on='train', suffixes=('', '_info'), how='left')
loco_plan = loco_plan.merge(loco_info, on='loco', suffixes=('', '_info'), how='left')
team_plan = team_plan.merge(team_info, on='team', suffixes=('', '_info'), how='left')
team_plan['team_type'] = team_plan.team.apply(lambda x: 'Реальная' if str(x)[0] == '2' else 'Фейковая')

In [7]:
def nice_time(t):
    return time.strftime(time_format, time.localtime(t)) if t > 0 else ''

In [17]:
st_name = 'ИРКУТСК-СОРТИРОВОЧНЫЙ'
team_plan['depot_name'] = team_plan.depot.map(st_names.name)
cols = ['team', 'number', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
team_plan[(team_plan.st_from_name == st_name) & (team_plan.state.isin([0, 1]))
          & (team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 24 * 3600)].sort_values('time_start')[cols]

team_plan[team_plan.number == 9205004408][cols]

,team,number,st_from_name,st_to_name,time_start_norm,time_end_norm,state,loco
1232,200200250998,9205004408,ИРКУТСК-СОРТИРОВОЧНЫЙ,ВОЕННЫЙ ГОРОДОК,"Jun 30, 21:18","Jun 30, 21:25",0,-1
1233,200200250998,9205004408,ВОЕННЫЙ ГОРОДОК,ИРКУТСК-ПАССАЖИРСКИЙ,"Jun 30, 21:25","Jun 30, 21:32",0,-1
1234,200200250998,9205004408,ИРКУТСК-ПАССАЖИРСКИЙ,КАЯ,"Jun 30, 22:12","Jun 30, 22:23",0,-1
1235,200200250998,9205004408,КАЯ,ГОНЧАРОВО,"Jun 30, 22:23","Jun 30, 22:37",0,-1
1236,200200250998,9205004408,ГОНЧАРОВО,БОЛЬШОЙ ЛУГ,"Jun 30, 22:39","Jun 30, 22:56",0,-1
1237,200200250998,9205004408,БОЛЬШОЙ ЛУГ,АНДРИАНОВСКАЯ,"Jun 30, 22:56","Jun 30, 23:55",0,-1
1238,200200250998,9205004408,АНДРИАНОВСКАЯ,АНГАСОЛКА,"Jun 30, 23:55","Jul 01, 00:09",0,-1
1239,200200250998,9205004408,АНГАСОЛКА,СЛЮДЯНКА II,"Jul 01, 00:09","Jul 01, 00:34",0,-1
1240,200200250998,9205004408,СЛЮДЯНКА II,СЛЮДЯНКА I,"Jul 01, 00:34","Jul 01, 00:44",0,-1


In [63]:
team_info['depot_name'] = team_info.depot.map(st_names.name)
team_info['in_plan'] = team_info.team.isin(team_plan[team_plan.state == 1].team)
team_info['oper_time_f'] = team_info.oper_time.apply(nice_time)
cols = ['team', 'number', 'depot_name', 'state']
a = team_info[(team_info.ttype == 1) & (team_info.loc_name == 'СЛЮДЯНКА I') 
          #& (team_info.depot_name.isin(['СЛЮДЯНКА I']))
          & (team_info.oper_time < current_time + 24 * 3600)]
a.depot_name.value_counts()

СЛЮДЯНКА I               97
ЗИМА                     17
ИРКУТСК-СОРТИРОВОЧНЫЙ    12
УЛАН-УДЭ                 11
Name: depot_name, dtype: int64

In [90]:
b = a[a.in_plan == True]
#cols = ['team', 'number', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
#team_plan[team_plan.team.isin(b.team)][cols]
cols = ['team', 'regions', 'number', 'depot_name', 'ready_type', 'depot_st', 'depot_time', 'return_st', 'oper_time_f', 'state']
b = a[(a.in_plan) & (a.regions.apply(lambda x: '2002118236' in x))].sort_values('oper_time')[cols]
print(b.team.count()) # всего 30 слюдянковских бригад, которые могут ездить до Иркутска
# Еще 29 бригад из Зимы и Иркутска - они тоже могут ехать в нечетную сторону
team_plan[(team_plan.team.isin(b.team)) & (team_plan.st_from_name == 'СЛЮДЯНКА I') & (team_plan.state.isin([0, 1]))].st_to_name.value_counts()

30


СЛЮДЯНКА II    29
БАЙКАЛЬСК       1
Name: st_to_name, dtype: int64

In [53]:
twr = pd.read_csv(FOLDER + 'team_work_region.csv')
twr['link'] = twr.link.apply(literal_eval)
twr['st_from_name'] = twr.link.apply(lambda x: x[0]).map(st_names.name)
twr['st_to_name'] = twr.link.apply(lambda x: x[1]).map(st_names.name)
twr[twr.twr == 2002118236]

,twr,link,st_from_name,st_to_name
2994,2002118236,"[2000037090, 2000036958]",КАЯ,ИРКУТСК-ПАССАЖИРСКИЙ
2995,2002118236,"[2000036958, 2000037090]",ИРКУТСК-ПАССАЖИРСКИЙ,КАЯ
2996,2002118236,"[2000037086, 2000036958]",ВОЕННЫЙ ГОРОДОК,ИРКУТСК-ПАССАЖИРСКИЙ
2997,2002118236,"[2000036958, 2000037086]",ИРКУТСК-ПАССАЖИРСКИЙ,ВОЕННЫЙ ГОРОДОК
2998,2002118236,"[2000037100, 2000037090]",ГОНЧАРОВО,КАЯ
2999,2002118236,"[2000037090, 2000037100]",КАЯ,ГОНЧАРОВО
3000,2002118236,"[2000036956, 2000037100]",ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО
3001,2002118236,"[2000037100, 2000036956]",ГОНЧАРОВО,ИРКУТСК-СОРТИРОВОЧНЫЙ
3002,2002118236,"[2000037100, 2000037132]",ГОНЧАРОВО,БОЛЬШОЙ ЛУГ
3003,2002118236,"[2000037132, 2000037100]",БОЛЬШОЙ ЛУГ,ГОНЧАРОВО


In [67]:
train_plan['train_type'] = train_plan.train.apply(lambda x: x[0])
train_plan[(train_plan.st_from_name == 'СЛЮДЯНКА I') & (train_plan.st_to_name == 'СЛЮДЯНКА II')
          & (train_plan.time_start >= current_time) & (train_plan.time_start < current_time + 24 * 3600)].train_type.value_counts()

# Всего 96 поездов в нечетную сторону из Слюдянки!!!
# Из них всего 15 локомотивов резервом и 81 настоящий поезд

2    80
8    15
9     1
Name: train_type, dtype: int64

In [ ]:
Итого на Слюдянку надо 96 бригад. А есть только 59 (на начало планирования)
Надо где-то найти еще 37. 
Еще 10 бригад едут от Иркутска в Слюдянку на начало планирования. Осталось 27.

In [91]:
cols = ['team', 'number', 'depot_name', 'depot_st', 'depot_time', 'state', 'loc_name', 'oper_time_f']
team_info['link'] = list(zip(team_info.st_from, team_info.st_to))
links = pd.read_csv(FOLDER + 'link.csv', dtype={'st_from':str, 'st_to':str})
links['link'] = list(zip(links.st_from, links.st_to))
team_info['dir'] = team_info.link.map(links.set_index('link')['dir'])
team_info[(team_info.depot_name.isin(['ЗИМА', 'ИРКУТСК-СОРТИРОВОЧНЫЙ'])) & (team_info.state.isin(['2','3','4']) == False)
         & (team_info['dir'] == 0)].sort_values('oper_time')[cols]

,team,number,depot_name,depot_st,depot_time,state,loc_name,oper_time_f
4086,200200252131,9203003471,ЗИМА,2000036710,1467214200,0,СЛЮДЯНКА I - БАЙКАЛЬСК,"Jun 30, 05:24"
4299,200200280412,9203008981,ЗИМА,2000036710,1467221700,0,СЛЮДЯНКА I - БАЙКАЛЬСК,"Jun 30, 07:31"
665,210203241335,9203026111,ЗИМА,2000036710,1467280800,1,ЗИМА - ЗАЛАРИ,"Jun 30, 13:54"
197,200200173784,9203004826,ЗИМА,2000036710,1467280200,1,ЗИМА - ЗАЛАРИ,"Jun 30, 14:04"
983,200253980740,9205031362,ИРКУТСК-СОРТИРОВОЧНЫЙ,2000036956,1467280800,1,ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО,"Jun 30, 14:07"
1074,200200186389,9203004450,ЗИМА,2000036710,1467282600,1,ЗИМА - ЗАЛАРИ,"Jun 30, 14:47"
296,200200142770,9203000487,ЗИМА,2000036710,1467285000,1,ЗИМА - ЗАЛАРИ,"Jun 30, 15:10"
1040,200200209907,9205007663,ИРКУТСК-СОРТИРОВОЧНЫЙ,2000036956,1467243900,1,ЗИМА - ЗАЛАРИ,"Jun 30, 15:19"
1229,200200280980,9205008378,ИРКУТСК-СОРТИРОВОЧНЫЙ,2000036956,1467286200,1,ИРКУТСК-СОРТИРОВОЧНЫЙ - ГОНЧАРОВО,"Jun 30, 15:22"
590,200200209842,9205001133,ИРКУТСК-СОРТИРОВОЧНЫЙ,2000036956,1467207000,1,КАСЬЯНОВКА - МАЛЬТА,"Jun 30, 15:22"


In [99]:
cols = ['team', 'number', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
team_plan[team_plan.team == '200200035170'][cols]

,team,number,st_from_name,st_to_name,time_start_norm,time_end_norm,state,loco
5513,200200035170,9203025216,ЗИМА,ЗАЛАРИ,"Jun 30, 16:00","Jun 30, 17:08",1,200200072108
5514,200200035170,9203025216,ЗАЛАРИ,ЧЕРЕМХОВО,"Jun 30, 17:08","Jun 30, 18:22",1,200200072108
5515,200200035170,9203025216,ЧЕРЕМХОВО,КАСЬЯНОВКА,"Jun 30, 18:22","Jun 30, 18:36",1,200200072108
5516,200200035170,9203025216,КАСЬЯНОВКА,МАЛЬТА,"Jun 30, 18:36","Jun 30, 19:14",1,200200072108
5517,200200035170,9203025216,МАЛЬТА,КИТОЙ,"Jun 30, 19:14","Jun 30, 19:46",1,200200072108
5518,200200035170,9203025216,КИТОЙ,АНГАРСК,"Jun 30, 19:46","Jun 30, 19:52",1,200200072108
5519,200200035170,9203025216,АНГАРСК,СУХОВСКАЯ,"Jun 30, 19:52","Jun 30, 19:59",1,200200072108
5520,200200035170,9203025216,СУХОВСКАЯ,МЕГЕТ,"Jun 30, 19:59","Jun 30, 20:12",1,200200072108
5521,200200035170,9203025216,МЕГЕТ,БАТАРЕЙНАЯ,"Jun 30, 20:12","Jun 30, 20:20",1,200200072108
5522,200200035170,9203025216,БАТАРЕЙНАЯ,ИРКУТСК-СОРТИРОВОЧНЫЙ,"Jun 30, 20:20","Jun 30, 20:28",1,200200072108


In [88]:
cols = ['team', 'number', 'st_from_name', 'st_to_name', 'time_start_norm', 'time_end_norm', 'state', 'loco']
team_plan[(team_plan.depot_name == st_name) & (team_plan.st_to_name == 'ГОНЧАРОВО')
          & (team_plan.time_start >= current_time) & (team_plan.time_start < current_time + 8 * 3600)
         & (team_plan.state == 1) & (team_plan.st_from_name == st_name)].sort_values('time_start')[cols]

,team,number,st_from_name,st_to_name,time_start_norm,time_end_norm,state,loco
32500,200200133888,9205004034,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 17:35","Jun 30, 18:11",1,200200090027
32286,200210521507,9205031257,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 18:26","Jun 30, 19:02",1,200200096438
34943,200200107130,9205003330,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 18:26","Jun 30, 19:02",1,200200083732
240,200200164060,9205003038,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 18:35","Jun 30, 19:11",1,200200058242
36950,200200124135,9205002662,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 18:45","Jun 30, 19:21",1,200200031392
33387,200200239976,9205002622,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 19:15","Jun 30, 19:51",1,200200089310
18546,200200137338,9205000565,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 19:45","Jun 30, 20:21",1,200200102514
3358,200200189739,9205007646,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 19:55","Jun 30, 20:31",1,200200093794
37242,200200209173,9205002451,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 20:05","Jun 30, 20:41",1,200200139161
21127,200200220843,9205000632,ИРКУТСК-СОРТИРОВОЧНЫЙ,ГОНЧАРОВО,"Jun 30, 20:15","Jun 30, 20:51",1,200200096700


In [105]:
team_info[(team_info.depot_st == '-1') & (team_info.depot_time == -1)].team.count() / team_info.team.count()

0.078265662318574319